In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

## Train

In [5]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)

for col in categorical_features:
    dt = df_train[col].dtype
    if dt == int or dt == float:
        df_train[col].fillna(-1, inplace=True)
        df_test[col].fillna(-1, inplace=True)
    else:
        df_train[col].fillna('', inplace=True)
        df_test[col].fillna('', inplace=True)
        
for col in categorical_features:
    if col not in ['pj_no']:
        vc = dict(df_train[col].value_counts())
        for k, v in vc.items():
            if v <= 3:
                df_train[col].replace(k, 'Low_freq', inplace=True)
                df_test[col].replace(k, 'Low_freq', inplace=True)
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col])
        mapping = {v: i for i, v in enumerate(le.classes_)}
        df_test[col] = df_test[col].apply(lambda x: mapping[x] if x in mapping else -1)
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

In [6]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01225944

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's l2: 0.0676625
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0669024
[3]	valid_0's l2: 0.0661908
[4]	valid_0's l2: 0.0654758
[5]	valid_0's l2: 0.0648214
[6]	valid_0's l2: 0.0641902
[7]	valid_0's l2: 0.0634881
[8]	valid_0's l2: 0.0628317
[9]	valid_0's l2: 0.0621559
[10]	valid_0's l2: 0.0615044
[11]	valid_0's l2: 0.060871
[12]	valid_0's l2: 0.0602352
[13]	valid_0's l2: 0.0596189
[14]	valid_0's l2: 0.0590747
[15]	valid_0's l2: 0.0584742
[16]	valid_0's l2: 0.0579309
[17]	valid_0's l2: 0.0573374
[18]	valid_0's l2: 0.0567615
[19]	valid_0's l2: 0.0561879
[20]	valid_0's l2: 0.0556353
[21]	valid_0's l2: 0.0551097
[22]	valid_0's l2: 0.054578
[23]	valid_0's l2: 0.0540893
[24]	valid_0's l2: 0.0536198
[25]	valid_0's l2: 0.0531203
[26]	valid_0's l2: 0.0526395
[27]	valid_0's l2: 0.0521627
[28]	valid_0's l2: 0.0516811
[29]	valid_0's l2: 0.0512135
[30]	valid_0's l2: 0.0507445
[31]	valid_0's l2: 0.0503195
[32]	valid_0's l2: 0.0498867
[33]	valid_0's l2: 0

[303]	valid_0's l2: 0.020677
[304]	valid_0's l2: 0.0206563
[305]	valid_0's l2: 0.0206392
[306]	valid_0's l2: 0.020629
[307]	valid_0's l2: 0.0206107
[308]	valid_0's l2: 0.0205959
[309]	valid_0's l2: 0.0205787
[310]	valid_0's l2: 0.0205616
[311]	valid_0's l2: 0.0205497
[312]	valid_0's l2: 0.0205325
[313]	valid_0's l2: 0.0205193
[314]	valid_0's l2: 0.0205
[315]	valid_0's l2: 0.0204805
[316]	valid_0's l2: 0.02046
[317]	valid_0's l2: 0.020443
[318]	valid_0's l2: 0.020425
[319]	valid_0's l2: 0.0204076
[320]	valid_0's l2: 0.0203913
[321]	valid_0's l2: 0.0203774
[322]	valid_0's l2: 0.0203667
[323]	valid_0's l2: 0.0203503
[324]	valid_0's l2: 0.0203335
[325]	valid_0's l2: 0.0203176
[326]	valid_0's l2: 0.0203022
[327]	valid_0's l2: 0.0202856
[328]	valid_0's l2: 0.0202707
[329]	valid_0's l2: 0.0202545
[330]	valid_0's l2: 0.0202414
[331]	valid_0's l2: 0.0202251
[332]	valid_0's l2: 0.0202079
[333]	valid_0's l2: 0.0201948
[334]	valid_0's l2: 0.0201819
[335]	valid_0's l2: 0.0201634
[336]	valid_0's l2:

[610]	valid_0's l2: 0.0181463
[611]	valid_0's l2: 0.0181428
[612]	valid_0's l2: 0.018139
[613]	valid_0's l2: 0.0181334
[614]	valid_0's l2: 0.0181292
[615]	valid_0's l2: 0.0181255
[616]	valid_0's l2: 0.018119
[617]	valid_0's l2: 0.0181171
[618]	valid_0's l2: 0.0181129
[619]	valid_0's l2: 0.0181064
[620]	valid_0's l2: 0.0181042
[621]	valid_0's l2: 0.0181004
[622]	valid_0's l2: 0.0180973
[623]	valid_0's l2: 0.0180947
[624]	valid_0's l2: 0.0180874
[625]	valid_0's l2: 0.0180856
[626]	valid_0's l2: 0.0180825
[627]	valid_0's l2: 0.01808
[628]	valid_0's l2: 0.0180771
[629]	valid_0's l2: 0.0180732
[630]	valid_0's l2: 0.0180682
[631]	valid_0's l2: 0.0180643
[632]	valid_0's l2: 0.0180596
[633]	valid_0's l2: 0.018056
[634]	valid_0's l2: 0.0180519
[635]	valid_0's l2: 0.0180473
[636]	valid_0's l2: 0.0180403
[637]	valid_0's l2: 0.018036
[638]	valid_0's l2: 0.0180316
[639]	valid_0's l2: 0.0180277
[640]	valid_0's l2: 0.018025
[641]	valid_0's l2: 0.0180208
[642]	valid_0's l2: 0.0180177
[643]	valid_0's l

[924]	valid_0's l2: 0.0174386
[925]	valid_0's l2: 0.0174392
[926]	valid_0's l2: 0.0174364
[927]	valid_0's l2: 0.0174357
[928]	valid_0's l2: 0.0174335
[929]	valid_0's l2: 0.0174323
[930]	valid_0's l2: 0.0174315
[931]	valid_0's l2: 0.0174289
[932]	valid_0's l2: 0.017429
[933]	valid_0's l2: 0.0174263
[934]	valid_0's l2: 0.0174259
[935]	valid_0's l2: 0.0174225
[936]	valid_0's l2: 0.0174221
[937]	valid_0's l2: 0.0174192
[938]	valid_0's l2: 0.0174183
[939]	valid_0's l2: 0.0174159
[940]	valid_0's l2: 0.0174153
[941]	valid_0's l2: 0.0174125
[942]	valid_0's l2: 0.0174108
[943]	valid_0's l2: 0.0174101
[944]	valid_0's l2: 0.017409
[945]	valid_0's l2: 0.0174065
[946]	valid_0's l2: 0.0174047
[947]	valid_0's l2: 0.0174036
[948]	valid_0's l2: 0.0174018
[949]	valid_0's l2: 0.0173989
[950]	valid_0's l2: 0.0173969
[951]	valid_0's l2: 0.0173974
[952]	valid_0's l2: 0.0173973
[953]	valid_0's l2: 0.0173962
[954]	valid_0's l2: 0.0173955
[955]	valid_0's l2: 0.0173948
[956]	valid_0's l2: 0.0173928
[957]	valid_

...

[1704]	valid_0's l2: 0.011807
[1705]	valid_0's l2: 0.011807
[1706]	valid_0's l2: 0.0118071
[1707]	valid_0's l2: 0.0118066
[1708]	valid_0's l2: 0.0118062
[1709]	valid_0's l2: 0.0118057
[1710]	valid_0's l2: 0.011805
[1711]	valid_0's l2: 0.0118051
[1712]	valid_0's l2: 0.0118035
[1713]	valid_0's l2: 0.0118036
[1714]	valid_0's l2: 0.0118035
[1715]	valid_0's l2: 0.0118034
[1716]	valid_0's l2: 0.0118028
[1717]	valid_0's l2: 0.0118023
[1718]	valid_0's l2: 0.0118017
[1719]	valid_0's l2: 0.0118014
[1720]	valid_0's l2: 0.0117987
[1721]	valid_0's l2: 0.0117991
[1722]	valid_0's l2: 0.0117975
[1723]	valid_0's l2: 0.011795
[1724]	valid_0's l2: 0.0117961
[1725]	valid_0's l2: 0.011796
[1726]	valid_0's l2: 0.0117963
[1727]	valid_0's l2: 0.0117974
[1728]	valid_0's l2: 0.0117968
[1729]	valid_0's l2: 0.0117981
[1730]	valid_0's l2: 0.0117972
[1731]	valid_0's l2: 0.0117972
[1732]	valid_0's l2: 0.011797
[1733]	valid_0's l2: 0.0117971
[1734]	valid_0's l2: 0.0117976
[1735]	valid_0's l2: 0.0117973
[1736]	valid_0

[1978]	valid_0's l2: 0.0117513
[1979]	valid_0's l2: 0.0117518
[1980]	valid_0's l2: 0.0117511
[1981]	valid_0's l2: 0.0117505
[1982]	valid_0's l2: 0.0117502
[1983]	valid_0's l2: 0.0117503
[1984]	valid_0's l2: 0.011748
[1985]	valid_0's l2: 0.0117471
[1986]	valid_0's l2: 0.0117467
[1987]	valid_0's l2: 0.0117457
[1988]	valid_0's l2: 0.0117469
[1989]	valid_0's l2: 0.0117457
[1990]	valid_0's l2: 0.0117447
[1991]	valid_0's l2: 0.0117455
[1992]	valid_0's l2: 0.0117446
[1993]	valid_0's l2: 0.0117451
[1994]	valid_0's l2: 0.0117451
[1995]	valid_0's l2: 0.0117435
[1996]	valid_0's l2: 0.0117434
[1997]	valid_0's l2: 0.0117429
[1998]	valid_0's l2: 0.0117418
[1999]	valid_0's l2: 0.0117415
[2000]	valid_0's l2: 0.011741
[2001]	valid_0's l2: 0.0117411
[2002]	valid_0's l2: 0.0117405
[2003]	valid_0's l2: 0.01174
[2004]	valid_0's l2: 0.0117397
[2005]	valid_0's l2: 0.0117394
[2006]	valid_0's l2: 0.0117397
[2007]	valid_0's l2: 0.0117397
[2008]	valid_0's l2: 0.0117391
[2009]	valid_0's l2: 0.0117378
[2010]	valid

[2257]	valid_0's l2: 0.0117187
[2258]	valid_0's l2: 0.011718
[2259]	valid_0's l2: 0.0117175
[2260]	valid_0's l2: 0.011718
[2261]	valid_0's l2: 0.0117176
[2262]	valid_0's l2: 0.0117167
[2263]	valid_0's l2: 0.0117158
[2264]	valid_0's l2: 0.0117154
[2265]	valid_0's l2: 0.0117156
[2266]	valid_0's l2: 0.0117154
[2267]	valid_0's l2: 0.0117162
[2268]	valid_0's l2: 0.0117155
[2269]	valid_0's l2: 0.0117154
[2270]	valid_0's l2: 0.0117152
[2271]	valid_0's l2: 0.0117149
[2272]	valid_0's l2: 0.0117153
[2273]	valid_0's l2: 0.0117153
[2274]	valid_0's l2: 0.0117153
[2275]	valid_0's l2: 0.0117151
[2276]	valid_0's l2: 0.0117153
[2277]	valid_0's l2: 0.011715
[2278]	valid_0's l2: 0.0117154
[2279]	valid_0's l2: 0.0117149
[2280]	valid_0's l2: 0.0117154
[2281]	valid_0's l2: 0.0117147
[2282]	valid_0's l2: 0.0117149
[2283]	valid_0's l2: 0.0117147
[2284]	valid_0's l2: 0.0117149
[2285]	valid_0's l2: 0.0117146
[2286]	valid_0's l2: 0.0117141
[2287]	valid_0's l2: 0.0117148
[2288]	valid_0's l2: 0.0117137
[2289]	vali

[2536]	valid_0's l2: 0.0117095
[2537]	valid_0's l2: 0.0117098
[2538]	valid_0's l2: 0.0117107
[2539]	valid_0's l2: 0.0117111
[2540]	valid_0's l2: 0.0117104
[2541]	valid_0's l2: 0.0117101
[2542]	valid_0's l2: 0.0117102
[2543]	valid_0's l2: 0.0117111
[2544]	valid_0's l2: 0.0117108
[2545]	valid_0's l2: 0.0117103
[2546]	valid_0's l2: 0.0117099
[2547]	valid_0's l2: 0.0117089
[2548]	valid_0's l2: 0.0117074
[2549]	valid_0's l2: 0.0117077
[2550]	valid_0's l2: 0.0117089
[2551]	valid_0's l2: 0.0117089
[2552]	valid_0's l2: 0.0117094
[2553]	valid_0's l2: 0.0117095
[2554]	valid_0's l2: 0.0117093
[2555]	valid_0's l2: 0.0117092
[2556]	valid_0's l2: 0.0117096
[2557]	valid_0's l2: 0.0117095
[2558]	valid_0's l2: 0.0117099
[2559]	valid_0's l2: 0.011709
[2560]	valid_0's l2: 0.0117087
[2561]	valid_0's l2: 0.0117077
[2562]	valid_0's l2: 0.0117071
[2563]	valid_0's l2: 0.0117064
[2564]	valid_0's l2: 0.0117057
[2565]	valid_0's l2: 0.0117053
[2566]	valid_0's l2: 0.0117058
[2567]	valid_0's l2: 0.0117055
[2568]	va

[2823]	valid_0's l2: 0.0117018
[2824]	valid_0's l2: 0.0117015
[2825]	valid_0's l2: 0.0117034
[2826]	valid_0's l2: 0.0117044
[2827]	valid_0's l2: 0.0117045
[2828]	valid_0's l2: 0.0117037
[2829]	valid_0's l2: 0.0117039
[2830]	valid_0's l2: 0.0117041
[2831]	valid_0's l2: 0.0117036
[2832]	valid_0's l2: 0.0117036
[2833]	valid_0's l2: 0.0117033
[2834]	valid_0's l2: 0.0117024
[2835]	valid_0's l2: 0.0117025
[2836]	valid_0's l2: 0.0117034
[2837]	valid_0's l2: 0.0117031
[2838]	valid_0's l2: 0.0117029
[2839]	valid_0's l2: 0.0117029
[2840]	valid_0's l2: 0.0117026
[2841]	valid_0's l2: 0.0117031
[2842]	valid_0's l2: 0.0117043
[2843]	valid_0's l2: 0.0117048
[2844]	valid_0's l2: 0.0117051
[2845]	valid_0's l2: 0.0117057
[2846]	valid_0's l2: 0.0117056
[2847]	valid_0's l2: 0.0117054
[2848]	valid_0's l2: 0.0117052
[2849]	valid_0's l2: 0.0117051
[2850]	valid_0's l2: 0.0117047
[2851]	valid_0's l2: 0.011704
[2852]	valid_0's l2: 0.0117041
[2853]	valid_0's l2: 0.0117045
[2854]	valid_0's l2: 0.0117046
[2855]	va

## Ridge

In [7]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

continue_features.remove('keiyaku_pr')
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.Ridge(alpha=2, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_ridge.tsv', sep='\t', header=None, index=False)

# 0.01409408

5-fold cv mean l2 0.01409408


## HuberRegressor

In [8]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.HuberRegressor(max_iter=500, epsilon=1.2, alpha=0.00001)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_huber.tsv', sep='\t', header=None, index=False)

# 0.01359192

5-fold cv mean l2 0.01359192


## Linear svr

In [9]:
from sklearn import svm

splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = svm.LinearSVR(C=0.1, epsilon=0.01, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_linear_svr.tsv', sep='\t', header=None, index=False)

# 0.01322624

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


5-fold cv mean l2 0.01322624


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Merge

In [10]:
df_1 = pd.read_csv('./submission.tsv', sep='\t', names=['id', 'pred'])     #0.01225944
df_2 = pd.read_csv('./submission_ridge.tsv', sep='\t', names=['id', 'pred'])   #0.01409408
df_3 = pd.read_csv('./submission_huber.tsv', sep='\t', names=['id', 'pred'])  #0.01359192
df_4 = pd.read_csv('./submission_linear_svr.tsv', sep='\t', names=['id', 'pred'])  #0.01322624

df_1['pred'] = df_1['pred']*0.7 + df_2['pred']*0.05 + df_3['pred']*0.1 + df_4['pred']*0.15
df_1.to_csv('submission_merge.tsv', sep='\t', index=False, header=None) 